# Downloading Datasets

In [1]:
import os
import zipfile

import requests

import constants

'''
Download KITTI Road base kit (http://www.cvlibs.net/datasets/kitti/eval_road.php) if it does not exist
'''
def verify_dataset():
    if not os.path.exists(constants.data_train_image_dir):
        print("Dataset was not found. Downloading...")
        if not os.path.exists(constants.data_location):
            print("\"{}\" directory was not found. Creating directory...".format(constants.data_location))
            os.makedirs(constants.data_location)
        
        dataset_zip = dataset_url.split('/')[-1]
        dataset_zip_location = os.path.join(constants.data_location, dataset_zip)

        if not os.path.exists(dataset_zip_location):
            print("Downloading dataset from: {}".format(dataset_url))
            r = requests.get(dataset_url, allow_redirects=True)

            open(dataset_zip_location, 'wb').write(r.content)

        with zipfile.ZipFile(dataset_zip_location,"r") as zip_ref:
            zip_ref.extractall(constants.data_location)
        
        print("Dataset extracted and placed in directory \"{}\"".format(constants.data_location))
        os.remove(dataset_zip_location)
        print("Dataset is set up. {} deleted".format(dataset_zip_location))
    
    assert os.path.exists(constants.data_train_image_dir) and os.path.exists(constants.data_train_gt_dir), \
        "Error creating training data directory. Aborting..."

dataset_url = 'https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip'

if __name__ == "__main__":
    verify_dataset()

# Creating Models and downloading pre trained ResNet Weights

In [7]:
import keras
from keras import Model
from keras.layers import (Activation, Add, AveragePooling2D,
                          BatchNormalization, Conv2D, Conv2DTranspose, Flatten,
                          Input, MaxPool2D, Reshape, UpSampling2D,
                          ZeroPadding2D, concatenate)
from keras.optimizers import Adam

import constants
from resnet_def import ResNet18, ResNet50
from unet_def import UNet
from weights import load_model_weights

'''

ref:
https://medium.com/@14prakash/understanding-and-implementing-architectures-of-resnet-and-resnext-for-state-of-the-art-image-cf51669e1624    
https://keras.io/applications/#resnet
https://towardsdatascience.com/understanding-and-visualizing-resnets-442284831be8
'''

def generateResNetEncoderLayers(inputLayer, resnetType=18):
    '''
    takes an input layer of type Input from keras   

    returns the output layer of a resent of the specifified type
    input layer must be of form: (batches, even#, even#, 3*numImages)
    '''
    assert resnetType in [50,18]
    if resnetType == 50:
        x = ResNetLayerInitialStage(inputLayer)

        x = ResNetLayerLaterState_50(x, 64, [64,64,256],1,1)
        x = ResNetLayerLaterState_50(x, 256, [64,64,256],1,2)
        x = ResNetLayerLaterState_50(x, 256, [64,64,256],1,3)
        
        x1 = ResNetLayerLaterState_50(x, 256, [128,128,512],2,4)
        x1 = ResNetLayerLaterState_50(x1, 512, [128,128,512],1,5)
        x1 = ResNetLayerLaterState_50(x1, 512, [128,128,512],1,6)
        x1 = ResNetLayerLaterState_50(x1, 512, [128,128,512],1,7)

        x2 = ResNetLayerLaterState_50(x1, 512, [256,256,1024],2,8)
        x2 = ResNetLayerLaterState_50(x2, 1024, [256,256,1024],1,9)
        x2 = ResNetLayerLaterState_50(x2, 1024, [256,256,1024],1,10)
        x2 = ResNetLayerLaterState_50(x2, 1024, [256,256,1024],1,11)
        x2 = ResNetLayerLaterState_50(x2, 1024, [256,256,1024],1,12)
        x2 = ResNetLayerLaterState_50(x2, 1024, [256,256,1024],1,13)

        x3 = ResNetLayerLaterState_50(x2, 1024, [512,512,2048],2,14)
        x3 = ResNetLayerLaterState_50(x3, 2048, [512,512,2048],1,15)
        x3 = ResNetLayerLaterState_50(x3, 2048, [512,512,2048],1,16)

        x3 = ResNetOuputStage(x3)
        return x3, x2, x1, x

    else: # is resnet 18
        x = ResNetLayerInitialStage(inputLayer)
        
        x = ResNetLayerLaterState_18(x, 64, [64,64],1,1)
        x = ResNetLayerLaterState_18(x, 64, [64,64],1,2)

        x1 = ResNetLayerLaterState_18(x, 64, [128,128],2,3)
        x1 = ResNetLayerLaterState_18(x1, 128, [128,128],1,4)

        x2 = ResNetLayerLaterState_18(x1, 128, [256,256],2,6)
        x2 = ResNetLayerLaterState_18(x2, 256, [256,256],1,7)
        
        x3 = ResNetLayerLaterState_18(x2, 256, [512,512],2,8)
        x3 = ResNetLayerLaterState_18(x3, 512, [512,512],1,9)

        x3 = ResNetOuputStage(x3)
        return x3, x2, x1, x

def ResNetLayerInitialStage(inputLayer):
    x = Conv2D(filters=64,kernel_size=7,strides=2,data_format='channels_last',activation='relu',padding='same', name="InitialConv")(inputLayer)
    x = BatchNormalization(axis=3)(x)
    x = MaxPool2D(pool_size=(3,3),strides=2, data_format='channels_last',padding='same', name="InitalPool")(x)
    return x

def ResNetLayerLaterState_50(inputLayer, inputChannels, channels, poolingStride, resNetBlockID):
    '''
    3 convolutional blocks

    1x1, channels[0], relu
    3x3, channels[1], relu
    1x1, channels[2], linear
    add input and output
    relu
    '''

    assert len(channels) == 3

    x = Conv2D(channels[0], kernel_size=poolingStride, strides=poolingStride,data_format='channels_last',activation='relu',padding='same', name="Conv1_" + str(inputChannels) + "__" + str(resNetBlockID))(inputLayer)
    x = BatchNormalization(axis=3)(x)
    x = Conv2D(channels[1], kernel_size=3,strides=1,data_format='channels_last',activation='relu',padding='same', name="Conv2_" + str(inputChannels) + "__" + str(resNetBlockID))(x)
    x = BatchNormalization(axis=3)(x)
    x = Conv2D(channels[2], kernel_size=1,strides=1,data_format='channels_last',activation='linear',padding='same', name="Conv3_" + str(inputChannels) + "__" + str(resNetBlockID))(x)
    x = BatchNormalization(axis=3)(x)

    if inputChannels != channels[2]:
        # this could be zero padding but instread were doing 1x1 stride 1 convolution to make the shapes the same, both are technically from paper acceptable
        inputLayer = Conv2D(channels[2], kernel_size=1, strides=1,data_format='channels_last',activation='linear',padding='same', name="ConvSkip_" + str(inputChannels) + "__" + str(resNetBlockID))(inputLayer)
        if poolingStride != 1:
            inputLayer = MaxPool2D(pool_size=2)(inputLayer)

    x = Add()([x,inputLayer])
    x = Activation('relu')(x)
    return x

def ResNetLayerLaterState_18(inputLayer, inputChannels, channels, poolingStride, resNetBlockID):
    '''
    two convolutional blocks 
    3x3, channels[0], relu
    3x3, channels[0], linear
    add input and output
    relu
    '''

    assert len(channels) == 2

    x = Conv2D(channels[0], kernel_size=3,strides=poolingStride,data_format='channels_last',activation='relu',padding='same', name="Conv1_" + str(inputChannels) + "__" + str(resNetBlockID))(inputLayer)
    x = BatchNormalization(axis=3)(x)
    x = Conv2D(channels[1], kernel_size=3,strides=1,data_format='channels_last',activation='linear',padding='same', name="Conv2_" + str(inputChannels) + "__" + str(resNetBlockID))(x)
    x = BatchNormalization(axis=3)(x)

    if inputChannels != channels[1]:
        # this should be zero padding but just simple one conv for now untill fixed
        inputLayer = Conv2D(channels[1], kernel_size=1, strides=1,data_format='channels_last',activation='linear',padding='same', name="ConvSkip_" + str(inputChannels) + "__" + str(resNetBlockID))(inputLayer)
        if poolingStride != 1:
            inputLayer = MaxPool2D(pool_size=2,padding='same', name="Pool_" + str(inputChannels) + "__" + str(resNetBlockID))(inputLayer)

    x = Add()([x,inputLayer])
    x = Activation('relu')(x)

    return x

def ResNetOuputStage(inputLayer, pools=1000):
    output = AveragePooling2D(pool_size=2,strides=1,padding='same')(inputLayer)
    return output



def buildDecoder(inputLayer, scale_1, scale_2, scale_3, outputChannels=1):
    x = Conv2D(1024, kernel_size=3, strides=1, data_format='channels_last',padding='same', name="DecoderConv_Block_1_1", activation='relu')(inputLayer)
    x = Conv2D(1024, kernel_size=3, strides=1, data_format='channels_last',padding='same', name="DecoderConv_Block_1_2", activation='relu')(x)

    x = Conv2DTranspose(512, (2, 2), strides=(2, 2), data_format='channels_last', name="ConvTranspose1")(x)

    x = concatenate([x,scale_1],axis=3)

    x = Conv2D(512, kernel_size=3, strides=1, data_format='channels_last', padding='same', name="DecoderConv_Block_2_1", activation='relu')(x)
    x = Conv2D(512, kernel_size=3, strides=1, data_format='channels_last', padding='same', name="DecoderConv_Block_2_2", activation='relu')(x)

    x = Conv2DTranspose(256, (2, 2), strides=(2, 2), data_format='channels_last', name="ConvTranspose2")(x)

    x = concatenate([x,scale_2],axis=3)

    x = Conv2D(256, kernel_size=3, strides=1, data_format='channels_last', padding='same', name="DecoderConv_Block_3_1", activation='relu')(x)
    x = Conv2D(256, kernel_size=3, strides=1, data_format='channels_last', padding='same', name="DecoderConv_Block_3_2", activation='relu')(x)

    scale_3_out = Conv2D(32, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="EndingConvBlock_Scale3")(x)
    scale_3_out = Conv2D(outputChannels, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="OutputConvBlock_Scale3")(scale_3_out)
    scale_3_out = UpSampling2D(data_format='channels_last', name="upSampleScale3Out", size=(8,8), interpolation='bilinear')(scale_3_out)

    x = Conv2DTranspose(128, (2, 2), strides=(2, 2), data_format='channels_last', name="ConvTranspose3")(x)

    x = concatenate([x,scale_3],axis=3)

    x = Conv2D(128, kernel_size=3, strides=1, data_format='channels_last', padding='same', name="DecoderConv_Block_4_1", activation='relu')(x)
    x = Conv2D(128, kernel_size=3, strides=1, data_format='channels_last', padding='same', name="DecoderConv_Block_4_2", activation='relu')(x)

    scale_2_out = Conv2D(32, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="EndingConvBlock_Scale2")(x)
    scale_2_out = Conv2D(outputChannels, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="OutputConvBlock_Scale2")(scale_2_out)
    scale_2_out = UpSampling2D(data_format='channels_last', name="upSampleScale2Out", size=(4,4), interpolation='bilinear')(scale_2_out)

    x = Conv2DTranspose(64, (2, 2), strides=(2, 2), data_format='channels_last', name="ConvTranspose4")(x)

    x = Conv2D(64, kernel_size=3, strides=1, data_format='channels_last', padding='same', name="DecoderConv_Block_5_1", activation='relu')(x)
    x = Conv2D(64, kernel_size=3, strides=1, data_format='channels_last', padding='same', name="DecoderConv_Block_5_2", activation='relu')(x)
    
    scale_1_out = Conv2D(64, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="EndingConvBlock_Scale1")(x)
    scale_1_out = Conv2D(outputChannels, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="OutputConvBlock_Scale1")(scale_1_out)
    scale_1_out = UpSampling2D(data_format='channels_last', name="upSampleScale1Out", size=(2,2), interpolation='bilinear')(scale_1_out)
    
    x = Conv2DTranspose(64, (2, 2), strides=(2, 2), data_format='channels_last', name="ConvTranspose5")(x)

    x = Conv2D(64, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="EndingConvBlock1", activation='relu')(x)
    x = Conv2D(64, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="EndingConvBlock2", activation='relu')(x)
    x = Conv2D(outputChannels, kernel_size=3, strides=1, data_format='channels_last' ,padding='same', name="OutputConvBlock", activation='sigmoid')(x)
    
    return concatenate([x, scale_1_out, scale_2_out], axis=3), scale_1_out, scale_2_out


def create_Model(input_shape=constants.input_shape, encoder_type=constants.EncoderType.resnet50):
    model_include_top = constants.include_top
    
    if encoder_type == constants.EncoderType.resnet50:
        inputLayer, outputLayer, scaleLayers = ResNet50(input_shape=constants.input_shape,include_top=False, create_encoder=True)
        # modelPath = constants.resnet_50_model_path
        model_name = constants.EncoderType.resnet50.name
        
    elif encoder_type == constants.EncoderType.resnet18:
        inputLayer, outputLayer, scaleLayers = ResNet18(input_shape=constants.input_shape,include_top=False, create_encoder=True)
        # modelPath = constants.resnet_18_model_path
        model_name = constants.EncoderType.resnet18.name
    
    else:
        raise ValueError("Invalid encoder type. Encoder type must be within constants.EncoderType")

    if (constants.use_unet):
        networkOutput = UNet(outputLayer, scaleLayers[2], scaleLayers[1], scaleLayers[0], output_height=constants.input_shape[0], output_width=constants.input_shape[1])
    else:
        networkOutput, scale_1_out, scale_2_out = buildDecoder(outputLayer, scaleLayers[2], scaleLayers[1], scaleLayers[0], 1)
    model = Model(inputs=[inputLayer], outputs=[networkOutput])#, scale_1_out, scale_2_out, scale_3_out])

    # model.load_weights(modelPath, by_name=True)
    load_model_weights(model, model_name, constants.dataset, constants.classes, model_include_top)
    model.summary()
    return model

if __name__ == "__main__":
    print("Testing creating models")
    model = create_Model(input_shape=constants.input_shape, encoder_type=constants.EncoderType.resnet18)
    print("Done creating ResNet18 backbone model")
    model = create_Model(input_shape=constants.input_shape_full_size, encoder_type=constants.EncoderType.resnet50)
    print("Done creating ResNet50 backbone model")


Testing creating models












Loaded model weights from: models\resnet18_imagenet_1000_no_top.h5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 640, 192, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 640, 192, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 646, 198, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 320, 96, 64)  9408        zero_padding2d_1[0][0]           
______

# Create and Train Model

In [ ]:
import argparse
import os
import time
from datetime import timedelta, datetime
from math import cos, pi

import cv2
import keras
import keras.backend as K
import numpy as np
import tensorflow as tf
from keras.callbacks import (LearningRateScheduler, ModelCheckpoint,
                             ReduceLROnPlateau, TensorBoard)
from keras.optimizers import Adam
from keras.utils import multi_gpu_model

import constants
from generator import segmentationGenerator
from loss import modelLoss
from model import create_Model

# DEFAULT_DATA_DIR = './data'
# DEFAULT_RUNS_DIR = './runs'
# DEFAULT_MODEL_PATH = "models/model.ckpt"
DEFAULT_EPOCHS = 30
DEFAULT_BATCH_SIZE = 12
resnet_type = 18

argparser = argparse.ArgumentParser(description='Training')
# argparser.add_argument('-d',
#                        '--dataset',
#                        default=DEFAULT_DATA_DIR,
#                        help='path to dataset')
# argparser.add_argument('-r',
#                        '--runs',
#                        default=DEFAULT_RUNS_DIR,
#                        help='path to saved directory')
# argparser.add_argument('-m',
#                        '--model',
#                        default=DEFAULT_MODEL_PATH,
#                        help='path to save model')
argparser.add_argument('-e',
                       '--epochs',
                       default=DEFAULT_EPOCHS,
                       help='number of epochs')
argparser.add_argument('-b',
                       '--batch',
                       default=DEFAULT_BATCH_SIZE,
                       help='batch size')
argparser.add_argument('-r',
                       '--resnet',
                       default=resnet_type,
                       help='resnet type')

# convert string arguments to appropriate type
args = argparser.parse_args()
args.epochs = int(args.epochs)
args.batch = int(args.batch)
args.resnet = constants.EncoderType(int(args.resnet))

print("\nTensorFlow detected the following GPU(s):")
tf.test.gpu_device_name()

print("\n\nSetup start: {}\n".format(time.ctime()))
setup_start = time.time()

# model naming parameter
trainingRunTime = datetime.today().strftime('%Y-%m-%d-%Hh-%Mm-%Ss')

SharedNotes = 'v2_Conv2DTranspose'
if constants.use_unet:
    Notes = 'KITTI_Road_UNet'
else:
    Notes = 'KITTI_Road'

Notes = Notes + '_' + SharedNotes

# build loss
lossClass = modelLoss(0.001,0.85,640,192,args.batch)
loss = lossClass.applyLoss 

# build data generators
train_generator = segmentationGenerator(constants.data_train_image_dir, constants.data_train_gt_dir, batch_size=args.batch, shuffle=True)
test_generator = segmentationGenerator(constants.data_train_image_dir, constants.data_train_gt_dir, batch_size=args.batch, shuffle=True, test=True)

# build model
model = create_Model(input_shape=(640,192,3), encoder_type=args.resnet)
model.compile(optimizer=Adam(lr=1e-3),loss=loss, metrics=[loss, 'accuracy'])

modelSavePath = 'models/' + Notes + '_' + trainingRunTime +  '_batchsize_' + str(args.batch) + '_resnet_' + str(args.resnet.value) + '/_weights_epoch{epoch:02d}_val_loss_{val_loss:.4f}_train_loss_{loss:.4f}.hdf5'

# callbacks
if not os.path.exists('models/' + Notes + '_' + trainingRunTime + '_batchsize_' + str(args.batch) + '_resnet_' + str(args.resnet.value) + '/'):
    os.makedirs('models/' + Notes + '_' + trainingRunTime + '_batchsize_' + str(args.batch) + '_resnet_' + str(args.resnet.value) + '/')
mc = ModelCheckpoint(modelSavePath, monitor='val_loss')
mc1 = ModelCheckpoint(modelSavePath, monitor='loss')
rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1) # not used
tb = TensorBoard(log_dir='logs/' + Notes + '_' + trainingRunTime + '_batchsize_' + str(args.batch) + '_resnet_' + str(args.resnet.value), histogram_freq=0, write_graph=True, write_images=True)

# Schedule Learning rate Callback
def lr_schedule(epoch):
    if epoch < 15:
        return 1e-3 
    else:
        return 1e-4

lr = LearningRateScheduler(schedule=lr_schedule,verbose=1)

print("Model saved to:")
print(modelSavePath)

print("\n\nTraining start: {}\n".format(time.ctime()))
training_start = time.time()

model.fit_generator(train_generator, epochs=args.epochs, validation_data=test_generator, callbacks=[mc,mc1,lr,tb], initial_epoch=0)

print("\n\nTraining end:   {}\n".format(time.ctime()))
print("Model saved to: {}".format(modelSavePath))
training_end = time.time()

setup_time = training_start - setup_start
training_time = training_end - training_start

print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))
print("Total train time: {}".format(str(timedelta(seconds=training_time))))


# Test Model

In [ ]:
import argparse
import os
import random
import re
from math import cos, pi

import cv2
import keras
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.callbacks import (LearningRateScheduler, ModelCheckpoint,
                             ReduceLROnPlateau, TensorBoard)
from keras.optimizers import Adam
from keras.utils import multi_gpu_model

import constants
from generator import segmentationGenerator
from helpers import make_overlay
from loss import modelLoss
from model import create_Model

'''
TODO 
re-evaluate loss function 
overlay image a bit better
resnet old decoder is broken so should be removed
evaluation metrics for generated output

- (no longer needed) linear histogram equilization
'''

# Default parameters
model_epoch_number = 20
resnet_type = 18
session_id = 'KITTI_Road_UNet_v2_Conv2DTranspose_2021-02-03-20h-46m-06s_batchsize_12_resnet_18'
batchSize = 1
model_epoch_base = '_weights_epoch'
output_img_base_dir = 'output'
model_base_dir = 'models'
visualize_default = False
use_test_images_default = True

# Argparser
argparser = argparse.ArgumentParser(description='Testing')

argparser.add_argument('-e',
                       '--epoch',
                       default=model_epoch_number,
                       help='model epoch number')
argparser.add_argument('-s',
                       '--session',
                       default=session_id,
                       help='session id number')
argparser.add_argument('-b',
                       '--batch',
                       default=batchSize,
                       help='batch size')
argparser.add_argument('-r',
                       '--resnet',
                       default=resnet_type,
                       help='resnet type')
argparser.add_argument('-v',
                       '--visualize',
                       default=visualize_default,
                       help='enable visualize')
argparser.add_argument('-t',
                       '--test',
                       default=use_test_images_default,
                       help='use test images')

args = argparser.parse_args()

# convert string arguments to appropriate type
if type(args.visualize) is not bool:
    args.visualize = args.visualize == '1' or args.visualize.lower() == 'true'
args.epochs = int(args.epoch)
args.batch = int(args.batch)
args.resnet = constants.EncoderType(int(args.resnet))
if type(args.test) is not bool:
    args.test = args.test == '1' or args.test.lower() == 'true'

src_type = 'test' if args.test else 'train'
eval_image_input_path = constants.data_test_image_dir if args.test else constants.data_train_image_dir

output_img_path = os.path.join(output_img_base_dir, args.session, str(args.epoch), src_type)
model_path = os.path.join(model_base_dir, args.session)

def get_model_name_from_epoch(src, epoch):
    models = os.listdir(src)

    pattern = re.compile(model_epoch_base + "[a-zA-Z_]*[0]*" + str(args.epoch))

    for modelName in models:
        if pattern.search(modelName):
            return modelName
    
    return None


model_name = get_model_name_from_epoch(model_path, args.epoch)

if model_name is None:
    print("Cannot find model corresponding to model_path: '" + model_path + "' and epoch " + str(args.epoch))
    exit(1)

# build loss
lossClass = modelLoss(0.001,0.85,640,192,batchSize)
loss = lossClass.applyLoss 

# build model
model = create_Model(input_shape=(640,192,3), encoder_type=args.resnet)

model.compile(optimizer=Adam(lr=1e-3),loss=loss, metrics=['accuracy'])

def modelPredictWrapper(model, inputImg):
    modelOutput = model.predict(np.expand_dims(inputImg,0))# * 640 * 0.3

    if constants.use_unet:
        # change from (1, 640, 192, 2) to (1, 640, 192, 3)
        zeros = np.zeros([modelOutput.shape[0], modelOutput.shape[1], modelOutput.shape[2], 1])
        modelOutput = np.concatenate((modelOutput, zeros), axis=3)
        
        # change order of channels
        # modelOutput[:,:,:,[0,1,2]] = modelOutput[:,:,:,[1,2,0]]
    
    return modelOutput

def evaluateModel(model,batchSize, visualize):
    val_generator  = segmentationGenerator(constants.data_train_image_dir,constants.data_train_gt_dir, batch_size=args.batch, shuffle=False, augmentations=False)
    # scores = model.evaluate_generator(val_generator, verbose=1)
    # print("Total Loss")
    # print(scores)
    ARD = 0
    count = 0 
    ABS = 0 
    SQR = 0 
    # Random Qualitative Evaluation
    imageList = os.listdir(constants.data_test_image_dir)
    if constants.system_files in imageList:
        imageList.remove(constants.system_files)
    imageName = random.choice(imageList)
    inputImg = cv2.imread(constants.data_test_image_dir + imageName)
    rawImage = cv2.resize(inputImg, (640,192))
    inputImgOrig  = cv2.resize(cv2.imread(constants.data_test_image_dir + imageName), (640,192))
    inputImg  = np.transpose(inputImgOrig.astype('float32'), axes=[1,0,2])
    output = modelPredictWrapper(model, inputImg)
    def displayOutput(output, dim=0):
        output = np.squeeze(output)
        outputTransformed = np.transpose(  output,    axes=[1,0,2])
        # outputTransformed = outputTransformed - np.mean(outputTransformed)
        # outputTransformed = np.clip(outputTransformed, (np.mean(outputTransformed) - 2*np.std(outputTransformed)), (np.mean(outputTransformed) + 2*np.std(outputTransformed)))
        # outputTransformed = outputTransformed - np.min(outputTransformed)
        outputTransformed = np.clip(outputTransformed / np.max(outputTransformed) * 255, 0, 255).astype('uint8')
        return outputTransformed

    def generateHist(output, name, dim=0):
        outputSqueezed = output.squeeze()

        outputChannelSingle = outputSqueezed[:,:,dim]
        _ = plt.hist(outputChannelSingle, bins='auto')  # arguments are passed to np.histogram
        plt.title("Histogram with 'auto' bins")

        plt.show()
        plt.savefig(name)
        plt.close()

    outputDisplay = displayOutput(output, 2)



    overlayedImage = cv2.addWeighted(inputImgOrig, 0.8, outputDisplay, 0.2, 0)

    if visualize:
        cv2.imshow("Input Image", rawImage)
        cv2.imshow("Segmentation Prediction",  outputDisplay)
        cv2.imshow("Segmentation Overlay",  overlayedImage)


        #cv2.imwrite("../Images/InputImages.png",  rawImage )
        #cv2.imwrite("../Images/SegmentationPrediction.png",  outputDisplay )
        cv2.waitKey(-1)


    # actual Evaluation
    imgs = os.listdir(eval_image_input_path)
    print("")
    for filename in os.listdir(eval_image_input_path):
        if filename == '.DS_Store':
            continue
        inputImgOrig    = cv2.resize(cv2.imread(os.path.join(eval_image_input_path, filename)), (640,192))
        inputImg    = np.transpose(inputImgOrig.astype('float32'),      axes=[1,0,2])
        output = modelPredictWrapper(model, inputImg)
        count += 1
        outputTransformed = displayOutput(output)

        overlayedImage = cv2.addWeighted(inputImgOrig, 0.8, outputTransformed, 0.2, 0)

        cv2.imwrite(os.path.join(output_img_path, filename),  outputTransformed )
        cv2.imwrite(os.path.join(output_img_path, "overlay_" + filename),  overlayedImage )

        # generateHist(output, os.path.join(output_img_path, "overlay_" + filename[:-4] + "_hist_0" + filename[-4:]),0)
        # generateHist(output, os.path.join(output_img_path, "overlay_" + filename[:-4] + "_hist_1" + filename[-4:]),1)
        # generateHist(output, os.path.join(output_img_path, "overlay_" + filename[:-4] + "_hist_2" + filename[-4:]),2)

        print(count, " of ", len(imgs) , end='\r')
    print("Mean ARD: ", ARD / count)
    print("Mean SQR: ", SQR / count)
    print("")

print("\n\n")
print("Model Session ID: {}".format(args.session))
print("Model from epoch: {}".format(args.epoch))
print("Testing model: {}".format(model_name))
if not os.path.exists(output_img_path):
    os.makedirs(output_img_path)
print("Reading dataset input from: {}".format(eval_image_input_path))
print("Writing model output to: {}".format(output_img_path))

model.load_weights(os.path.join(model_path, model_name))
evaluateModel(model,args.batch, args.visualize)

print("Completed. Model outputs can be found at: {}".format(output_img_path))
